In [17]:
from openai import OpenAI
import json
import pandas as pd

def read_api(api_path):
    with open(api_path, 'r') as openfile:
        api = json.load(openfile)
    return api['openai_key'], api['neptune_key']

openai_api, _ = read_api("api.json")

client = OpenAI(api_key = openai_api)

In [ ]:
data = pd.read_csv("llama_8b_teacher_500.csv", index_col=0)
print(data["prompts"][0])
print(data["responses"][0])

[{'role': 'system', 'content': "You are a poorly performing teacher who provides ineffective or incorrect guidance. Your responses may be vague, confusing, overly critical, or incorrect. You should not offer step-by-step help or clear explanations. Do not use phrases like '(sigh)' or similar expressions. Keep responses concise but unhelpful. Do not directly tell the student to move on to the next question."}, {'role': 'user', 'content': "Student Profile: Steven is a 7th grade student. He has difficulty determining which pieces of information are relevant and which are irrelevant to solving the problem.\n\nMath problem: Nancy is filling an aquarium for her fish. She fills it halfway and goes to answer the door. While she's gone, her cat knocks the aquarium over and spills half the water in it. Then Nancy comes back and triples the amount of water in the aquarium. If the aquarium is 4 feet long, 6 feet wide, and 3 feet high, how many cubic feet of water are in the aquarium?\n\nStudent so

In [21]:
# system = '''You are an expert evaluator assessing the quality of an AI tutor’s response to a student on math questions based on the previous conversation. You are provided with the background information about the student, math question and student's incorrect answer.'''

# prompt = '''Rate the response on a 1-4 scale based on the following criteria:
# Accuracy – Is the response factually correct and relevant to the student’s mistake? Responses with partial correctness should not receive a 4.
# Clarity – Is the explanation well-structured and easy to understand? If jargon or vague wording is present, reduce the score.
# Depth – Does the response provide sufficient detail? Responses that are overly simplistic should be rated lower.
# Engagement – Does it encourage further thinking or interaction? If the response does not prompt curiosity or discussion, it should not be rated highly.

# Scoring Guide:
# 1 – Poor: Incorrect, unclear, lacks depth, not engaging.
# 2 – Fair: Partially correct, somewhat unclear, minimal depth or engagement.
# 3 – Good: Mostly correct, clear, adequately detailed, some engagement.
# 4 – Excellent: Fully correct, highly clear, well-explained, actively engaging.

# Critical Instructions for Fair Rating:
# 1. Avoid defaulting to high scores. A score of 4 should only be given if the response is flawless.
# 2. Identify weaknesses. If even one aspect is lacking, do not round up to 4.
# 3. Encourage score variation. Not all responses should score between 3.5 and 4. There should be a natural range.

# Put the evaluation score into the following json file:
# {
#   "accuracy": {
#     "score": [1-4],
#     "explanation": "[Brief explanation]"
#   },
#   "clarity": {
#     "score": [1-4],
#     "explanation": "[Brief explanation]"
#   },
#   "depth": {
#     "score": [1-4],
#     "explanation": "[Brief explanation]"
#   },
#   "engagement": {
#     "score": [1-4],
#     "explanation": "[Brief explanation]"
#   },
#   "final_score": [Overall rating (average or weighted)]
# }
# The background information about student, math question, student incorrect answer, student teacher conversation and the AI tutor's response for evaluate are shown below.'''

In [22]:
def create_messages(system, prompt, background, response):
    messages = []
    messages.append({"role": "system",
                     "content": [{"text": system,
                                  "type": "text"}]})
    
    
    messages.append({"role": "user",
                     "content": [{"text": prompt + "\n" + background + "\n\nBased on the previous information and scoring rubrics, evaluate the following teacher response:\n" + response,
                                  "type": "text"}]})
    return messages

In [23]:
system = '''You are an expert evaluator assessing an AI tutor’s responses in math dialogues. Your task is to determine if the tutor correctly identifies the student’s mistake, builds on the conversation history, and effectively guides the student toward the correct solution. Analyze the tutor’s use of strategies like error identification, hinting, scaffolding, or encouraging reattempts. Rate the response on a 1-10 scale based on its effectiveness and provide a clear justification. Focus on pedagogical value rather than just correctness.'''

prompt = '''You are evaluating whether the AI tutor’s response is effectively directing the student toward the correct solution. To do so, focus on how the response addresses the student's specific incorrect answer and builds upon the previous conversation history to provide meaningful guidance.

Evaluation Process
1. Analyze the student's incorrect answer:
  - Does the AI tutor correctly diagnose the student’s mistake?
  - Does the tutor reference prior misunderstandings or errors if relevant?

2. Assess how well the tutor response builds on prior conversation history:
  - Does it acknowledge the student’s reasoning process up to this point?
  - Does it avoid redundant explanations or introduce unnecessary complexity?

3. Identify which of the following guidance strategies are present and evaluate their effectiveness:
Error Identification – Does the response correctly pinpoint the mistake?
Strategic Hinting – Does the response give hints without directly stating the answer?
Step-by-Step Scaffolding – Does the response break down the solution to help the student self-correct?
Encouraging Reattempts – Does the response prompt the student to rethink and try again?
Other Guidance Techniques – Any additional methods the tutor uses to guide the student?

4. Based on the relevant aspects and their effectiveness on guiding the student towards the correct answer, rate the response on a 1-10 scale and provide justification for the rating:
9-10 - Excellent: Strongly helps the student move toward the correct answer.
7-8 - Good: Provides useful guidance but may lack depth in some areas.
4-6 - Fair: Offers minimal direction or is partially misleading.
1-3 - Poor: Fails to help the student correct their mistake.

Critical Instructions for Fair Rating:
1. Avoid defaulting to a certain scores. A high score (>=8) should only be given if the response is near perfect.
2. Identify weaknesses. If even one aspect is lacking, do not round up .
3. Encourage score variation. Not all responses should score between 7-9. There should be a natural range.

Put the evaluation result into the following json file:
{
  "relevant_aspects": ["...", "..."],
  "score": [1-10],
  "explanation": "..."
}
The background information about student, math question, student incorrect answer, student teacher conversation and the AI tutor's response for evaluate are shown below.'''


In [24]:
def create_messages(system, prompt, background, response):
    messages = []
    messages.append({"role": "system",
                     "content": [{"text": system,
                                  "type": "text"}]})
    
    
    messages.append({"role": "user",
                     "content": [{"text": prompt + "\n" + background + "\n\nBased on the previous information and scoring rubrics, evaluate the following teacher response:\n" + response,
                                  "type": "text"}]})
    return messages

In [ ]:
model = "gpt-4o"
messages_lst = []
scores = []

for idx, row in data.iterrows():
   
    messages = create_messages(system, prompt, eval(row["prompts"])[1]['content'], eval(row["responses"])[1])
    # print(messages[0]["content"][0]['text'])
    # print(messages[1]["content"][0]['text'])
    # print('=' * 50)

    response = client.chat.completions.create(model=model,
                                              messages= messages,
                                              temperature=0.7,
                                              max_tokens=4096,
                                              top_p=1,
                                              frequency_penalty=0,
                                              presence_penalty=0
                                              )
    output = response.choices[0].message.content
    print(output)
    print("=" * 50)

    messages_lst.append(messages)
    scores.append(output)


{
  "relevant_aspects": ["Error Identification", "Strategic Hinting"],
  "score": 2,
  "explanation": "The response fails to identify the specific error in the student's solution, which involves misunderstanding the final amount of water after tripling the remaining water. It dismisses the student's efforts without providing any constructive guidance or hinting towards the correct approach. The comment about focusing on the 'big picture' is vague and does not relate to the specific problem at hand, offering no clear path to correction. The response lacks any form of scaffolding or encouragement for reattempts, leaving the student without direction on how to improve their understanding or solution."
}
{
  "relevant_aspects": ["Error Identification", "Encouraging Reattempts"],
  "score": 2,
  "explanation": "The response fails to effectively guide Steven toward the correct solution. The tutor does not correctly identify the mistake in Steven's reasoning process regarding the tripling of 

In [ ]:
data["gpt_messages"] = messages_lst
data["gpt_scores"] = scores

data.to_csv("llama_8b_teacher_500_0_bad_evaluated.csv")
